# Sitzung 3


Diese Skripte sind ausschließlich als Zusatz-Material gedacht. Speziell für diejenigen unter Euch, die einen Einblick in das Programmieren gewinnen wollen. Wenn Du es also leid bist repetitive Tätigkeiten auszuführen und das lieber einer Maschine überlassen willst, bist Du hier genau richtig. 

<span style="color:red"> Die Codes sind nicht für die Klausur relevant, genau genommen haben sie mit dem Lehrstuhl für Statistik __rein gar nichts__ zu tun. </span>

Dieses mal lösen wir mal keine von den Übungsaufgaben. Stattdessen möchte ich Dir ein paar Einsichten aus der Vorlesung visuell unterfüttern und Dir gleichzeitig demonstrieren, was Jupyter Notebooks alles können. 

Also, zum Punkt: was haben wir vor? Wir wollen 
* die Gestalt der Binomialverteilung, der Poissonverteilung und der Normalverteilung kennenlernen, 
* eine Intuition für ihre Parameter entwickeln und 
* sie miteinander vergleichen.

Und weil einfache Schaubilder langweilig sind, machen wir unsere *interaktiv*.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact

Nachdem wir die Module importiert haben, die wir brauchen werden, definieren wir zunächst die Wahrscheinlichkeitsfunktionen der Binomialverteilung und der Poissonverteilung sowie die Dichtefunktion der Normalverteilung.

Zur Erinnerung:
* Binomialverteilung $X \sim Bin(n; p): \quad f_B (x, n, p) = {N \choose x} p^x \cdot (1-p)^{n-x}$

* Poissonverteilung $X \sim Pois(\lambda): \quad f_{PO}(x, \lambda) = \frac{\lambda^x}{x !} \cdot \exp(-\lambda)$

* Normalverteilung $X \sim N(\mu, \sigma): \quad f_N (x, \mu, \sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left( -\frac{(x-\mu)^2}{2 \sigma^2} \right)$

In [2]:
def pbinom(x, p, n):
    """Berechne die Wahrscheinlichkeit, dass X=x, für X ~ Binom(n, p)"""
    binom_coeff = (np.math.factorial(n)) / (np.math.factorial(x) * np.math.factorial(n - x))
    return binom_coeff * p**x * (1-p)**(n-x)

def ppois(x, l):
    """Berechne die Wahrscheinlichkeit, dass X=x, for X ~ Pois(l)"""
    return l**x / np.math.factorial(x) * np.exp(-l)

def pnorm(x, mu, sigma):
    """Berechne die Wahrscheinlichkeit, dass X<=x for X ~ N(mu, sigma)"""
    return 1/np.sqrt(2*np.pi*sigma**2) * np.exp(-0.5 * ((x - mu) / sigma)**2)

Als nächstes wollen wir eine Funktion definieren, die in der Lage ist die jeweiligen Verteilungen zu plotten. Und um es uns einfach zu machen, lassen wir nur die Lageparameter der Binomialverteilung als frei wählbare Argumente zu. Die anderen Parameter berchnen wir abhängig von denen der Binomialverteilung.

Den Parameter $\lambda$ der Poissonverteilung setzen wir gleich dem Erwartungswert der Binomialverteilung.

$$
\begin{align}
\lambda &= E\left[ X \right], \quad mit \ X \sim Bin(n, p) \\
        &= n \cdot p
\end{align}
$$

Ähnlich gehen wir bei den Parametern der Normalverteilung vor. Den Lageparameter setzen wir, wie oben, gleich dem Erwartungswert der Binomialverteilung. Den Streuungsparameter $\sigma$ bestimmen wir auch möglichst so, dass er der Binomialverteilung gleicht. Wir wissen, dass 

$$
\begin{align*}
Var(X) &= n \cdot p \cdot (1-p), \quad mit \ X \sim Bin(n, p) \\
\Rightarrow \sigma &= \sqrt{Var(X)} \\
                   &= \sqrt{n \cdot p \cdot (1-p)}
\end{align*}
$$

In [3]:
def view_pdf(p, n, binomial=True, poisson=False, normal=False):
    X_ = np.array(range(n+1))
        
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
    w = 0.5
    
    if binomial:
        y_binom = [pbinom(x, p, n) for x in X_]
        ax.bar(X_- w/2, y_binom, width=w, label="X ~ Bin({}, {})".format(n, p))
    
    if poisson:
        # Erwartungswert der Binomialverteilung wird der Lageparameter für die Poissonverteilung
        l = n * p
        y_pois = [ppois(x, l) for x in X_]
        ax.bar(X_+ w/2, y_pois, width=w, label="X ~ Pois({:3.2})".format(l))
    
    if normal:
        mu = n * p
        sigma = np.sqrt(n * p * (1-p))
        Xnorm = np.linspace(0, n, num=1000)
        Ynorm = pnorm(Xnorm, mu, sigma)
        ax.plot(Xnorm, Ynorm, c="grey", label="X ~ N({:3.2}, {:3.2})".format(mu, sigma), linewidth=5)
    
    plt.title("Vergleich verschiedener Verteilungen")
    plt.ylabel(r"$f(x)$")
    plt.xlabel(r"$X=x$")
    plt.legend()
    plt.show()

Jetzt geht es nur noch darum, unsere Funktion interaktiv aufzurufen, dazu verwenden wir die `interact`-Funktion aus dem `ipywidgets`-Modul.

Der übergeben wir unsere hausgemachte Funktion und können über die Argumente bestimmen, wie das user-interface (UI) aussehen soll. Das tun wir, indem wir der `interact`-Funktion Argumente übergeben, die denselben Namen haben, wie die Argumente unserer selbstgemachten Funktion. Der Datentyp des Arguments bestimmt die Art des UI-Elements. `Tupel` führen zu slidern, `Listen` führen zu dropdown-Menüs, Und `Booleans` führen zu checkboxen.

Für `n` wollen wir einen slider der nur ganze Zahlen zulässt, die größer sind als 0 (sie sollten auch kleiner als 150 sein, aber das hat numerische Gründe). Dazu übergeben wir ein Tupel, dass wiederum  aus `int`s besteht - dem Python Datentyp für ganze Zahlen. Die erste Position steht für den minimalen Wert des sliders, die zweite für den maximalen Wert und der dritte steht für die Schrittweite.

Für `p` wollen wir einen slider der Dezimalzahlen zwischen 0 und 1 erlaubt. Das funktioniert analog, nur dass unser Tupel jetzt aus `floats` besteht - dem Python-Datentyp für Dezimalzahlen.

In [4]:
interact(view_pdf, n=(1, 100, 1), p=(0.0, 1.0, 0.001), normal=False);

interactive(children=(FloatSlider(value=0.5, description='p', max=1.0, step=0.001), IntSlider(value=50, descri…